Script to process buttressing data for ismip7

created by ronja.reese@northumbria.ac.uk

In [2]:
import scipy.io
import xarray as xr
import numpy as np
import pandas as pd

In [3]:
data_dir = '/media/NAS2/ISMIP7/'
file_name = data_dir+'BFRN/ButtressingFluxResponseNumbers_withLatLon_2025_08.mat'

In [7]:
bfrn_mat = scipy.io.loadmat(file_name, squeeze_me=True)
bfrn_mat.pop('__header__')
bfrn_mat.pop('__version__')
bfrn_mat.pop('__globals__')

d = {
    "coords": {
        "x": {"dims": ("x"), "data": bfrn_mat['x'][0,:], "attrs": {"units": "m"}},
        "y": {"dims": ("y"), "data": bfrn_mat['y'][:,0], "attrs": {"units": "m"}}
    },
    "dims": ("x", "y"),
    "data_vars": {
        "BFRN": {"dims": ("y", "x"), "data": bfrn_mat['BFRN']},
    },
}

bfrn_ds = xr.Dataset.from_dict(d)

In [25]:
#bfrn_ds['BFRN'].plot.hist()

In [26]:
# Load BedMap2 ice shelf mask dataset on ISMIP grid
#bedmap_ismip = xr.open_dataset('../../pism-ais/pism_input_data/bedmap2/bedmap2_initmip8km.nc')
bedmap_ismip = xr.open_dataset(data_dir+'/share_with_modellers/topography/bedmap3_ismip_8km.nc')

In [27]:
# This is just a trick as the toography dataset lacks x,y coordinate values
xy_coords = xr.open_dataset(data_dir+'/ismip_grid/ismip_8km_60m_grid.nc')
bedmap_ismip = bedmap_ismip.assign_coords(x=xy_coords.x.values)
bedmap_ismip = bedmap_ismip.assign_coords(y=xy_coords.y.values)

In [32]:
# Interpolate BFRN onto ISMIP6 grid, using Bedmap2 mask
BFRN_interp = bfrn_ds['BFRN'].interp_like(bedmap_ismip['floating_frac'], method='nearest')


In [33]:
BFRN_interp = BFRN_interp.where(bedmap_ismip['floating_frac']> 0.5, np.nan)

In [34]:
BFRN_interp.assign_attrs(description='BRFN are given for each square (0 in non-floating regions and where response is smaller than the threshold 0.02%). BFRN = R/P with R the response in flux integrated over all grounding lines and P the perturbation strength for each square. Multiply by 100 to get the unit %. Assuming that the flux responds linearly to the perturbations, the change in flux across the grounding line resulting from a perturbation of X [Gt] over an area that lies within the square p is given by X × BFRN(p) [Gt/yr]. The response to a perturbation over larger areas is given as the sum of the perturbation within each square.')

<xarray.DataArray 'BFRN' (y: 761, x: 761)> Size: 5MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(761, 761))
Coordinates:
  * y        (y) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * x        (x) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
    lat      (y, x) float64 5MB ...
    lon      (y, x) float64 5MB ...
Attributes:
    description:  BRFN are given for each square (0 in non-floating regions a...

In [35]:
# Add BFRN bins 
binning_var = BFRN_interp.values.flatten()[np.logical_and(~np.isnan(BFRN_interp.values.flatten()), BFRN_interp.values.flatten()!=0 )]

In [36]:
bin_edges = pd.qcut(binning_var, q=9 , retbins=True)[1] 

In [37]:
bin_edges[0]= 0.00002 # replace with min value
bin_edges = np.insert(bin_edges, 0, 0)   # Add 0 in front
print("Bin edges: ", bin_edges)

Bin edges:  [0.00000000e+00 2.00000000e-05 3.08739845e-04 4.91103690e-04
 7.53561600e-04 1.14960296e-03 1.76281427e-03 2.84275194e-03
 5.51171067e-03 1.42970487e-02 7.43431494e+00]


In [38]:
bfrn_bins = BFRN_interp.copy(deep=True)*np.nan
for i,bin_edge in enumerate(bin_edges):
    bfrn_bins = bfrn_bins.where(bin_edge > BFRN_interp, i)

In [39]:
bfrn_bins = bfrn_bins.where(bfrn_bins<10, np.nan) 

In [40]:
BFRN_interp = xr.Dataset(dict(BFRN=BFRN_interp))

In [41]:
BFRN_interp['BFRN_bins'] = bfrn_bins

In [42]:
# Add median per bin
bfrn_medians = BFRN_interp['BFRN'].groupby_bins(BFRN_interp['BFRN'], bins=bin_edges).median()
bfrn_medians[0] = 0
bfrn_medians = bfrn_medians.to_dataset().assign(BFRN_bins=('dims', bfrn_medians.BFRN_bins.values))
bfrn_medians = bfrn_medians.rename({'BFRN': 'BFRN_medians'})
nfrn_medians = bfrn_medians.set_coords('BFRN_bins')
bfrn_medians = bfrn_medians.rename({'BFRN_bins': 'BFRN_bins_dim'})
bfrn_medians = bfrn_medians.drop_vars('BFRN_bins_dim')

In [44]:
BFRN_interp = xr.merge([BFRN_interp, bfrn_medians ],compat='override') 

In [45]:
# Add overall median of >0 values
BFRN_interp['BFRN_median'] = BFRN_interp['BFRN'].where(BFRN_interp['BFRN']>0, np.nan).median()

In [47]:
# Create NetCDF dataset and save
BFRN_interp.to_netcdf(data_dir+'BFRN/BFRN_ismip8km_v2.nc')